# 🏔️ Strava Dynamic Tables + AI Demo

This notebook creates Dynamic Tables with AI-powered insights using Strava-like activity data.

## What This Does:
1. Creates dedicated schema and warehouse for Dynamic Tables
2. Builds two Dynamic Tables with Cortex AI integration
3. Demonstrates automated ETL with incremental refresh
4. Generates AI-powered performance insights and athlete profiles

## Prerequisites:
1. Run `00_setup_environment.sql` first
2. Run `01_data_streaming_simulator.ipynb` to create and populate the ACTIVITIES table

## Use Cases Covered:
- Real-time activity processing with AI insights
- Athlete performance tracking with incremental updates
- AI-powered coaching recommendations
- Automated data pipeline without orchestration


## Setup Context
First, let's set up our Snowflake environment and context.


In [ ]:
-- Set up context
USE ROLE STRAVA_DEMO_ADMIN;
USE WAREHOUSE STRAVA_DEMO_WH;
USE DATABASE STRAVA_DEMO_SAMPLE;
USE SCHEMA RAW_DATA;


## Environment Setup
Create a dedicated schema and warehouse for Dynamic Tables to separate workloads.


In [ ]:
-- Create a dedicated schema for Dynamic Tables
CREATE SCHEMA IF NOT EXISTS STRAVA_DYNAMIC_TABLES;
USE SCHEMA STRAVA_DYNAMIC_TABLES;

-- Create warehouse for Dynamic Tables (separate from regular workloads)
CREATE OR REPLACE WAREHOUSE STRAVA_DT_DEMO_WH
    WAREHOUSE_SIZE = XSMALL
    AUTO_SUSPEND = 300
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = FALSE;

-- Grant permissions
GRANT USAGE ON WAREHOUSE STRAVA_DT_DEMO_WH TO ROLE STRAVA_DEMO_ADMIN;

SELECT 'Dynamic Tables environment setup completed!' as STATUS;


## Dynamic Table 1: Activity Intelligence
This Dynamic Table processes activities in real-time with AI-powered performance insights.

**Key Features:**
- 2-minute LAG for near real-time processing
- Cortex COMPLETE for AI-generated performance insights
- Real-time activity metrics calculation


In [ ]:
-- DT 1: Real-time Activity Intelligence (CDC + AI)
CREATE OR REPLACE DYNAMIC TABLE activity_intelligence
    LAG = '2 MINUTES'
    WAREHOUSE = STRAVA_DT_DEMO_WH
AS
    SELECT 
        activity_id,
        athlete_id,
        activity_type,
        start_date_local,
        distance_meters,
        moving_time_sec,
        elapsed_time_sec,
        total_elevation_gain_meters,
        average_heartrate,
        -- Calculate pace (km/h)
        CASE 
            WHEN moving_time_sec > 0 THEN (distance_meters / 1000.0) / (moving_time_sec / 3600.0)
            ELSE 0 
        END as pace_kmh,
        -- 🤖 CORTEX AI: Generate activity insights
        SNOWFLAKE.CORTEX.COMPLETE('llama3-8b',
            'Analyze this ' || activity_type || ' activity: ' ||
            ROUND(distance_meters/1000.0, 1) || 'km in ' || 
            ROUND(moving_time_sec/60.0, 0) || ' minutes. ' ||
            'Pace: ' || ROUND((distance_meters / 1000.0) / (moving_time_sec / 3600.0), 1) || 'km/h. ' ||
            'Provide a brief performance insight in 1-2 sentences.'
        ) as ai_performance_insight,
        CURRENT_TIMESTAMP() as processed_at
    FROM RAW_DATA.ACTIVITIES
    -- Only process recent activities (simulates streaming)
    WHERE start_date_local >= DATEADD('day', -30, CURRENT_TIMESTAMP())
;


## Dynamic Table 2: Athlete Performance Dashboard
This Dynamic Table aggregates athlete metrics and provides AI-powered insights for coaching and performance analysis.

**Key Features:**
- 5-minute LAG for aggregated KPIs
- Performance tier classification
- AI athlete profiling with Cortex COMPLETE
- Performance trend analysis with AI recommendations


In [ ]:
-- DT 2: Athlete Performance Intelligence (Aggregated KPIs + AI Insights)
CREATE OR REPLACE DYNAMIC TABLE athlete_performance_dashboard
    LAG = '5 MINUTES'
    WAREHOUSE = STRAVA_DT_DEMO_WH
AS
    SELECT 
        athlete_id,
        COUNT(*) as total_activities_7d,
        SUM(distance_meters) / 1000.0 as total_distance_km_7d,
        AVG(distance_meters) / 1000.0 as avg_distance_km,
        SUM(moving_time_sec) / 3600.0 as total_moving_hours_7d,
        AVG(pace_kmh) as avg_pace_kmh,
        AVG(average_heartrate) as avg_heartrate,
        SUM(total_elevation_gain_meters) as total_elevation_7d,
        -- Traditional performance tiers
        CASE 
            WHEN AVG(pace_kmh) > 15 THEN 'High Performer'
            WHEN AVG(pace_kmh) > 10 THEN 'Regular Athlete'
            ELSE 'Casual User'
        END as performance_tier,
        -- CORTEX AI: Athlete profiling with SQL
        SNOWFLAKE.CORTEX.COMPLETE('llama3-8b',
            'Based on this athlete data: ' ||
            total_activities_7d || ' activities, ' ||
            ROUND(total_distance_km_7d, 1) || 'km total distance, ' ||
            ROUND(avg_pace_kmh, 1) || 'km/h average pace, ' ||
            ROUND(avg_heartrate, 0) || ' average heart rate. ' ||
            'Provide a brief athlete profile and training recommendations in 2-3 sentences.'
        ) as ai_athlete_profile,
        MAX(start_date_local) as last_activity_date,
        CURRENT_TIMESTAMP() as metrics_updated_at
    FROM activity_intelligence
    GROUP BY athlete_id
;


## View Dynamic Tables
Let's examine what we've created and see the Dynamic Tables in action.


In [ ]:
-- List all Dynamic Tables created
SHOW DYNAMIC TABLES IN SCHEMA STRAVA_DYNAMIC_TABLES;



In [ ]:
-- Preview the main Dynamic Tables
SELECT * FROM activity_intelligence LIMIT 5;


In [ ]:
SELECT * FROM athlete_performance_dashboard LIMIT 3;


## Demo Complete! 🎉

You've successfully created a Dynamic Tables pipeline enhanced with Cortex AI for real-time activity processing and intelligent insights.

### What You Created:
✅ `ACTIVITY_INTELLIGENCE` - Real-time activity processing with AI insights (2-min LAG)  
✅ `ATHLETE_PERFORMANCE_DASHBOARD` - Aggregated athlete KPIs with AI profiles (5-min LAG)  
✅ Automated refresh pipeline with dependency management  

### Next Steps:
1. Use **03_monitoring_queries.sql** to monitor refresh history and view results
2. Go back to **01_data_streaming_simulator.ipynb** and stream more data
3. Watch Dynamic Tables auto-refresh and process new activities
4. Experiment with different LAG settings to balance freshness vs. cost
